<a href="https://colab.research.google.com/github/OmarMohammed88/E-Psychiatrist_model/blob/main/Models/TFIDF_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [45]:
!unzip archive.zip

Archive:  archive.zip
  inflating: tweet_emotions.csv      


In [79]:
import pandas as pd
df_1 = pd.read_csv("/content/emotion_dataset.txt",header=None,sep=';')
df_1.columns = ['text','label']

In [2]:
import pandas as pd
df_2 = pd.read_csv("/content/tweet_emotions.csv")
df_2.columns = ['tweet_id','label','text']

In [3]:
import re ,html

url_regexes = [
    r"(http(s)?:\/\/.)?(www\.)?[-a-zA-Z0-9@:%._\+~#=]{2,256}\.[a-z]{2,6}\b([-a-zA-Z0-9@:%_\+.~#?&//=]*)",
    r"@(https?|ftp)://(-\.)?([^\s/?\.#-]+\.?)+(/[^\s]*)?$@iS",
    r"http[s]?://[a-zA-Z0-9_\-./~\?=%&]+",
    r"www[a-zA-Z0-9_\-?=%&/.~]+",
    r"[a-zA-Z]+\.com",
    r"(?=http)[^\s]+",
    r"(?=www)[^\s]+",
    r"://",
]

user_mention_regex = r"@[\w\d]+"
email_regexes = [r"[\w-]+@([\w-]+\.)+[\w-]+", r"\S+@\S+"]
redundant_punct_pattern = (
    r"([!\"#\$%\'\(\)\*\+,\.:;\-<=·>?@\[\\\]\^_ـ`{\|}~—٪’،؟`୍“؛”ۚ【»؛\s+«–…‘]{2,})"
)
regex_tatweel = r"(\D)\1{2,}"
rejected_chars_regex = r"[^0-9\a-zA-Z\[\]!\"#\$%\'\(\)\*\+,\.:;\-<=·>?@\[\\\]\^_ـ`{\|}~—٪’،؟`୍“؛”ۚ»؛\s+«–…‘]"

regex_url_step1 = r"(?=http)[^\s]+"
regex_url_step2 = r"(?=www)[^\s]+"
regex_url = r"(http(s)?:\/\/.)?(www\.)?[-a-zA-Z0-9@:%._\+~#=]{2,256}\.[a-z]{2,6}\b([-a-zA-Z0-9@:%_\+.~#?&//=]*)"
regex_mention = r"@[\w\d]+"
regex_email = r"\S+@\S+"

chars_regex = r"0-9\u0621-\u063A\u0640-\u066C\u0671-\u0674a-zA-Z\[\]!\"#\$%\'\(\)\*\+,\.:;\-<=·>?@\[\\\]\^_ـ`{\|}~—٪’،؟`୍“؛”ۚ»؛\s+«–…‘"

white_spaced_double_quotation_regex = r'\"\s+([^"]+)\s+\"'
white_spaced_single_quotation_regex = r"\'\s+([^']+)\s+\'"
white_spaced_back_quotation_regex = r"\`\s+([^`]+)\s+\`"
white_spaced_em_dash = r"\—\s+([^—]+)\s+\—"

left_spaced_chars = r" ([\]!#\$%\),\.:;\?}٪’،؟”؛…»·])"
right_spaced_chars = r"([\[\(\{“«‘*\~]) "
left_and_right_spaced_chars = r" ([\+\-\<\=\>\@\\\^\_\|\–]) "
replace_urls_emails_mentions=True
strip_tashkeel=True
strip_tatweel=True
insert_white_spaces=True
remove_elongation=True
remove_html_markup=True


def preprocess(text):
    """
    Preprocess takes an input text line an applies the same preprocessing used in AraBERT
                        pretraining
    Args:
        text (:obj:`str`): inout text string
    Returns:
        string: A preprocessed string depending on which model was selected
    """


    text = str(text).lower()

    if replace_urls_emails_mentions:
        # replace all possible URLs
        for reg in url_regexes:
            text = re.sub(reg, " ", text)
        # REplace Emails with [بريد]
        for reg in email_regexes:
            text = re.sub(reg, " ", text)
        # replace mentions with [مستخدم]
        text = re.sub(user_mention_regex, " ", text)        

    if remove_html_markup:
        # remove html line breaks
        text = re.sub("<br />", " ", text)
        # remove html markup
        text = re.sub("</?[^>]+>", " ", text)


    text = re.sub(rejected_chars_regex, " ", text)

    # remove extra spaces
    text = " ".join(text.replace("\uFE0F", "").split())

    # ALl the other models dont require Farasa Segmentation

    return text

def _remove_redundant_punct(text):
    text_ = text
    result = re.search(redundant_punct_pattern, text)
    dif = 0
    while result:
        sub = result.group()
        sub = sorted(set(sub), key=sub.index)
        sub = " " + "".join(list(sub)) + " "
        text = "".join(
            (text[: result.span()[0] + dif], sub, text[result.span()[1] + dif :])
        )
        text_ = "".join(
            (text_[: result.span()[0]], text_[result.span()[1] :])
        ).strip()
        dif = abs(len(text) - len(text_))
        result = re.search(redundant_punct_pattern, text_)
    text = re.sub(r"\s+", " ", text)
    return text.strip()

In [4]:
df_2.text = df_2.text.apply(preprocess)

In [83]:
merged_data = pd.concat([df_1,df_2[['text','label']]],ignore_index=True)

In [84]:
merged_data.label = merged_data.label.map({"neutral":"neutral","worry":"worry","sadness":"sadness","happiness":"happiness","love":"love","surprise":"surprise",
                              "fun":"fun", "relief":"relief" ,"hate":"hate" ,"empty":"empty","enthusiasm":"enthusiasm","joy":"happiness","anger":"anger","fear":"worry"
                              ,"boredom":"boredom"}.get)

In [ ]:
merged_data.label.value_counts()

In [94]:
merged_data = merged_data[merged_data['text'].astype(bool)]


In [96]:
merged_data.to_csv("emotion_dataset.csv",index=False)

#TFIDF

In [5]:
import os

import pandas as pd
import numpy as np
from scipy.stats import randint
import seaborn as sns # used for plot interactive graph. 
import matplotlib.pyplot as plt
import seaborn as sns
from io import StringIO
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import chi2
from IPython.display import display
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC ,SVC
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn import metrics
#import warnings

In [6]:
train = pd.read_csv("train.txt.zip",sep=';',header=None)
train.columns = ['text','label']

val = pd.read_csv("val.txt",sep=';',header=None)
val.columns = ['text','label']


test = pd.read_csv("test.txt",sep=';',header=None)
test.columns = ['text','label']
df = pd.concat([train,val,test],ignore_index=True)
df

,text,label
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned...,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplac...,love
4,i am feeling grouchy,anger
...,...,...
19995,i just keep feeling like someone is being unki...,anger
19996,im feeling a little cranky negative after this...,anger
19997,i feel that i am useful to my people and that ...,joy
19998,im feeling more comfortable with derby i feel ...,joy


In [7]:
# df = pd.read_csv("/content/emotion_dataset.txt",sep=";",header=None)
# df.columns =['text','label']
class_name = list(df.label.unique())

dec = {}
for i in range(len(class_name)):
  dec[class_name[i]]=i

df.label = df.label.map(dec)
df

,text,label
0,i didnt feel humiliated,0
1,i can go from feeling so hopeless to so damned...,0
2,im grabbing a minute to post i feel greedy wrong,1
3,i am ever feeling nostalgic about the fireplac...,2
4,i am feeling grouchy,1
...,...,...
19995,i just keep feeling like someone is being unki...,1
19996,im feeling a little cranky negative after this...,1
19997,i feel that i am useful to my people and that ...,5
19998,im feeling more comfortable with derby i feel ...,5


In [8]:
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=3,
                        ngram_range=(1, 2), 
                        stop_words='english')

# We transform each complaint into a vector
features = tfidf.fit_transform(df.text).toarray()
corpus = ["i feel sad and depressed and don't want to live anymore"]
features_sen = tfidf.transform(corpus)
labels = df.label

print("Each of the %d complaints is represented by %d features (TF-IDF score of unigrams and bigrams)" %(features.shape))

Each of the 20000 complaints is represented by 10185 features (TF-IDF score of unigrams and bigrams)


In [10]:
N = 3
for Product, category_id in sorted(df.items()):
  features_chi2 = chi2(features, labels == category_id)
  indices = np.argsort(features_chi2[0])
  feature_names = np.array(tfidf.get_feature_names())[indices]
  unigrams = [v for v in feature_names if len(v.split(' ')) == 1]
  bigrams = [v for v in feature_names if len(v.split(' ')) == 2]
  print("\n==> %s:" %(Product))
  print("  * Most Correlated Unigrams are: %s" %(', '.join(unigrams[-N:])))
  print("  * Most Correlated Bigrams are: %s" %(', '.join(bigrams[-N:])))

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)



==> label:
  * Most Correlated Unigrams are: folks, follow, zumba
  * Most Correlated Bigrams are: feeling mournful, feeling naughty, feeling melancholy

==> text:
  * Most Correlated Unigrams are: folks, follow, zumba
  * Most Correlated Bigrams are: feeling mournful, feeling naughty, feeling melancholy


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [11]:
X = df['text']
y = df['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.25,
                                                    random_state = 0)

In [12]:
# models = [
#     LinearSVC(),
#     LogisticRegression(random_state=0),
#     SVC()
# ]

# # 5 Cross-validation
# CV = 5
# cv_df = pd.DataFrame(index=range(CV * len(models)))

# entries = []
# for model in models:
#   model_name = model.__class__.__name__
#   accuracies = cross_val_score(model, features, labels, scoring='accuracy', cv=CV)
#   for fold_idx, accuracy in enumerate(accuracies):
#     entries.append((model_name, fold_idx, accuracy))
    
# cv_df = pd.DataFrame(entries, columns=['model_name', 'fold_idx', 'accuracy'])

In [13]:
# mean_accuracy = cv_df.groupby('model_name').accuracy.mean()
# std_accuracy = cv_df.groupby('model_name').accuracy.std()

# acc = pd.concat([mean_accuracy, std_accuracy], axis= 1, 
#           ignore_index=True)
# acc.columns = ['Mean Accuracy', 'Standard deviation']
# acc

In [14]:
# plt.figure(figsize=(8,5))
# sns.boxplot(x='model_name', y='accuracy', 
#             data=cv_df, 
#             color='lightblue', 
#             showmeans=True)
# plt.title("MEAN ACCURACY (cv = 5)\n", size=14);

In [15]:
X_train, X_test, y_train, y_test,indices_train,indices_test = train_test_split(features, 
                                                               labels, 
                                                               df.index, test_size=0.25, 
                                                               random_state=1)



model = LinearSVC()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [16]:
# Classification report
print('\t\t\t\tCLASSIFICATIION METRICS\n')
print(metrics.classification_report(y_test, y_pred, 
                                    target_names= [str(i) for i in df['label'].unique()]))

				CLASSIFICATIION METRICS

              precision    recall  f1-score   support

           0       0.94      0.95      0.95      1444
           1       0.91      0.91      0.91       688
           2       0.85      0.75      0.79       427
           3       0.80      0.72      0.76       179
           4       0.88      0.86      0.87       559
           5       0.90      0.94      0.92      1703

    accuracy                           0.90      5000
   macro avg       0.88      0.85      0.87      5000
weighted avg       0.90      0.90      0.90      5000



In [17]:
from sklearn.metrics import f1_score
f1_score(y_test,y_pred,average='macro')

0.8660562475207553

In [41]:
dec

{'anger': 1, 'fear': 4, 'joy': 5, 'love': 2, 'sadness': 0, 'surprise': 3}

In [43]:
def infernece(text):
  text_tfidf = tfidf.transform([text])
  result = model.predict(text_tfidf)
  inverse_dec = {1:'anger', 4:'fear', 5:'joy',2: 'love',0:'sadness',3:'surprise'}
  return inverse_dec[result[0]]
  

In [57]:
infernece(" the demon said, her voice dripping sarcasm. Also known as passion, I said quietly. Passion has overthrown tyrants and freed prisoners and slaves. Passion has brought justice where there was savagery. Passion has created freedom where there was nothing but fear. Passion has helped souls rise from the ashes of their horrible lives and build something better, stronger, more beautiful")

'joy'